In [ ]:
import requests
import pandas as pd
import snowflake.connector
!pip install snowflake-connector-python

Grab the API with all of the continent data available.

In [16]:
response = requests.get('https://restcountries.com/v3.1/all')
countries_data = response.json()

Create a connection with snowflake and indicate which warehouse should be used.

In [ ]:
conn = snowflake.connector.connect(
    user='hedgehog',
    password='______', #redacted my password
    account='QLB09115',
    warehouse='ANIMAL_TASK_WH',
    database='MENTAL_HEALTH_HUB',
    schema='DATASETS'
)
cursor = conn.cursor()
cursor.execute("USE WAREHOUSE ANIMAL_TASK_WH")

Grab the query and write the result to a dataframe. Make a copy in case there are any errors in the manipulation process. 

In [ ]:
query = """
SELECT DISTINCT dp.code AS ID,
                dp.entity AS COUNTRY
FROM MENTAL_HEALTH_HUB.DATASETS.BY_DISORDER_PREVALENCE dp 
FULL JOIN INDICATOR_MH_POLICY e ON dp.entity = e.entity 
WHERE dp.code IS NOT NULL  
AND dp.entity NOT LIKE '%IHME GBD%'
"""
df = pd.read_sql(query, conn)
conn.close()
print(df)
dfa = df.copy()

Make a map for each continent. 

In [33]:
continent_map = {}
for country in countries_data:
    country_name = country.get('name', {}).get('common', '')
    country_code = country.get('cca3', '')  # or 'cca3' for 3-letter codes
    continent = country.get('region', '')  # or 'subregion' for more specific regions
    if country_code and continent:
        continent_map[country_code] = continent

Map the continent on to the country data frame and write the amended data frame to a csv file. 

In [36]:
dfa['CONTINENT'] = dfa['ID'].map(continent_map)
dfa.to_csv('countries_continents.csv', index=False)